In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as sm

In [57]:
df=pd.read_excel("KTMNewData.xlsx")

In [58]:
df.head()

,Age,Gender,Occupation,Phone_Type,Current_Bike,Relationship,Response
0,53,Male,Professional,Average,180 to 220,Complicated,Not purchased
1,27,Female,Self Employed,Low End,No Bike,Single,Purchased
2,39,Female,Unemployed,Average,180 to 220,Married,Not purchased
3,20,Female,Unemployed,High End,No Bike,Married,Not purchased
4,29,Male,Student,Average,180 to 220,Complicated,Purchased


In [59]:
df.info

<bound method DataFrame.info of       Age  Gender     Occupation Phone_Type Current_Bike Relationship  \
0      53   Male    Professional    Average   180 to 220  Complicated   
1      27  Female  Self Employed    Low End      No Bike       Single   
2      39  Female     Unemployed    Average   180 to 220      Married   
3      20  Female     Unemployed   High End      No Bike      Married   
4      29   Male         Student    Average   180 to 220  Complicated   
...   ...     ...            ...        ...          ...          ...   
1513   19   Male         Student   High End   125 to 180    Committed   
1514   18   Male         Student   High End    Below 125      Married   
1515   24  Female        Student   High End   180 to 220      Married   
1516   23  Female        Student   High End   125 to 180       Single   
1517   18   Male         Student   High End   180 to 220       Single   

           Response  
0     Not purchased  
1         Purchased  
2     Not purchased  
3  

In [60]:
for col in df.columns:
    if df[col].dtype!='object':
        continue
    elif df[col].dtype=='object':
        print(df[col].value_counts())

Female    816
Male      702
Name: Gender, dtype: int64
Student          686
Self Employed    295
Unemployed       274
Professional     263
Name: Occupation, dtype: int64
High End    836
Low End     343
Average     339
Name: Phone_Type, dtype: int64
180 to 220       420
125 to 180       333
Below 125        300
No Bike          254
220 and Above    211
Name: Current_Bike, dtype: int64
Single         490
Complicated    384
Committed      331
Married        313
Name: Relationship, dtype: int64
Purchased        872
Not purchased    646
Name: Response, dtype: int64


In [66]:
def LabelEncode(df_local):
    for col in df_local.columns:
        col_dict = {}
        if df_local[col].dtype!='object':
            continue
        elif df_local[col].dtype=='object':
            col2=df_local[col].unique()
            z=0
            for i in col2:
                col_dict[i]=z
                z+=1
            df_local[col]=df_local[col].map(col_dict)
            
    return df_local

df_encode = LabelEncode(df)

In [67]:
df_encode.head()

,Age,Gender,Occupation,Phone_Type,Current_Bike,Relationship,Response
0,53,1,1,1,1,1,0
1,27,2,2,2,2,2,1
2,39,2,3,1,1,3,0
3,20,2,3,3,2,3,0
4,29,1,4,1,1,1,1


In [94]:
from sklearn.model_selection import train_test_split
X = df_encode[['Age', 'Gender', 'Occupation', 'Phone_Type','Current_Bike','Relationship']]
Y = df_encode[['Response']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=40)
df_train=pd.concat([X_train, Y_train], axis=1).reindex(X_train.index)
df_test=pd.concat([X_test, Y_test], axis=1).reindex(X_test.index)

In [95]:
df_train.head()
df_train.size

7434

In [96]:
df_test.head()
df_test.size

3192

In [76]:
log_reg = sm.logit("Response~Age+Gender+Phone_Type+Occupation+Current_Bike",data=df_train).fit()

Optimization terminated successfully.
         Current function value: 0.460389
         Iterations 6


In [77]:
log_reg.summary()
# log(p/1-p)

#p close to 0 no
#p close to 1 yes

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Response   No. Observations:                 1062
Model:                          Logit   Df Residuals:                     1056
Method:                           MLE   Df Model:                            5
Date:                Sat, 05 Feb 2022   Pseudo R-squ.:                  0.3276
Time:                        16:07:21   Log-Likelihood:                -488.93
converged:                       True   LL-Null:                       -727.13
Covariance Type:            nonrobust   LLR p-value:                9.925e-101
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.4652      0.503      2.912      0.004       0.479       2.451
Age             -0.0754      0.007    -10.886      0.000      -0.089      -0.062
Gender          -0.6010      0.163     -3.678      0.000      -0.921      -0.281
Phone_Type       0.3947      0.101      3.902      0.000       0.196       0.593
Occupation       0.6059      0.077      7.857      0.000       0.455       0.757
Current_Bike    -0.1177      0.054     -2.161      0.031      -0.225      -0.011
================================================================================
"""

In [79]:
#Transformed = 1/(1+e^-a)
Y_predict=log_reg.predict(X_test)
prediction = list(map(round, Y_predict))

In [92]:

from sklearn.metrics import (confusion_matrix, accuracy_score)
 
# confusion matrix
cm = confusion_matrix(Y_test, prediction)
cm=pd.DataFrame(cm)
print ("Confusion Matrix : \n", cm)
 
# accuracy score of the model
Test_accuracy = accuracy_score(Y_test, prediction)
print('Test accuracy = ', Test_accuracy*100,"%")

Confusion Matrix : 
      0    1
0  130   54
1   54  218
Test accuracy =  76.31578947368422 %
